# **Mask Object Detection - Training**

## **Environment**

### ** Required libs **

In [ ]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow tensorboard tensorflow-estimator tensorboard-plugin-wit
!pip install tensorflow-gpu==1.14.0 tensorboard==1.14.0 tensorflow-estimator==1.14.0 

In [ ]:
!pip install -q pillow lxml jupyter matplotlib cython pandas contextlib2
!apt-get install -qq protobuf-compiler
!pip install -q pycocotools tf_slim

### **Configurations**

In [ ]:
import os

# Repo URL
repo_url = 'https://github.com/marcocelia/mask-object-detection'

# Models
MODELS_CONFIG = {
    'ssd_inception_v2_coco': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'model_path': '/models/tf1/my_ssd_inception_v2_coco/',
        'pipeline_file': 'pipeline.config'
    }
}

# Select a model to use.
selected_model = 'ssd_inception_v2_coco'

model_name = MODELS_CONFIG[selected_model]['model_name']
model_path = MODELS_CONFIG[selected_model]['model_path']
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Set Repository Home Directory
base_dir = '.'
repo_dir_path = os.path.abspath(os.path.join(base_dir, os.path.basename(repo_url)))

# Set Label Map (.pbtxt) path and pipeline.config path
label_map_pbtxt_fname = repo_dir_path + '/annotations/label_map.pbtxt'
pipeline_fname = repo_dir_path + model_path + pipeline_file

# Set .record path
test_record_fname = repo_dir_path + '/annotations/test.record'
train_record_fname = repo_dir_path + '/annotations/train.record'

# Set output directories and clean up
model_dir = repo_dir_path + '/training/'
output_dir = repo_dir_path + '/exported-models/'

### **Clone TF model repo**

In [ ]:
%cd /content
!git clone --quiet -b r1.13.0 https://github.com/tensorflow/models.git

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install .

# Test
!python object_detection/builders/model_builder_test.py

### **Download pretrained model**

In [ ]:

%cd /content/models/research
%rm -rf pretrained_model/

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = model_name + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model/'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(model_name, DEST_DIR)

# Check downloaded files
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

# Set fine tune checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
print("fine_tune_checkpoint: ", fine_tune_checkpoint)

### **Clone project repository**

In [ ]:
import os
%cd /content

# Clean up
!rm -rf {repo_dir_path}

# Clone
!git clone {repo_url}

# # Pull (just in case the repo already exists)
%cd {repo_dir_path}
!git pull

# # Check if label map and pipeline files exist
assert os.path.isfile(label_map_pbtxt_fname), '`{}` not exist'.format(label_map_pbtxt_fname)
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

!rm -rf {model_dir} {output_dir}
os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

## **Prepare dataset**

### **Split train/test**

In [ ]:
%cd {repo_dir_path}

# Split images to train:test = 9:1
!python scripts/partition_dataset.py -x -i images/ -r 0.1

# Check test images
!ls images/test

### **xml to csv**

In [ ]:
# Create train data:
!python scripts/xml_to_csv.py -i images/train -o annotations/train_labels.csv

# Create test data:
!python scripts/xml_to_csv.py -i images/test -o annotations/test_labels.csv

### **Create TF record**

In [ ]:
!cat annotations/label_map.pbtxt

In [ ]:
# Create train data:
!python scripts/generate_tfrecord_v1.py \
    --csv_input=annotations/train_labels.csv \
    --output_path=annotations/train.record \
    --img_path=images/train \
    --label_map annotations/label_map.pbtxt

# Create test data:
!python scripts/generate_tfrecord_v1.py \
    --csv_input=annotations/test_labels.csv \
    --output_path=annotations/test.record \
    --img_path=images/test \
    --label_map annotations/label_map.pbtxt

In [ ]:
# Check
assert os.path.isfile(test_record_fname), '`{}` not exist'.format(test_record_fname)
assert os.path.isfile(train_record_fname), '`{}` not exist'.format(train_record_fname)

## **Training**

### **Tensorboard**

In [ ]:
# Set log directory for tensorboard to watch
LOG_DIR = model_dir

# Clean up the directory
!rm -rf {LOG_DIR}/*
!ls -lrt {LOG_DIR}

In [ ]:
# Use magic command to launch tensorboard within the notebook
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

### **Train**

In [ ]:
%cd {repo_dir_path}
!git pull

In [ ]:
%cd {repo_dir_path}
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr